In [1]:
from langchain_community.retrievers import BM25Retriever
from anthropic import Anthropic
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatAnthropic, ChatOllama, ChatOpenAI
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS, Chroma
import keyring
from pykrx import stock     # pip install pykrx
from datetime import datetime

# api key
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
ANTHROPIC_API_KEY = keyring.get_password('anthropic', 'key_for_windows')

In [14]:
data = [
    {
        "기업명": "삼성전자",
        "날짜": "2024-03-02",
        "문서 카테고리": "인수합병",
        "요약": "삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.",
        "주요 이벤트": ["기업 인수합병"]
    }, {
        "기업명": "삼성전자",
        "날짜": "2024-03-24",
        "문서 카테고리": "인수합병",
        "요약": "테스트 하나 둘 셋",
        "주요 이벤트": ["신제품 출시"]
    }, {
        "기업명": "현대차",
        "날짜": "2024-04-02",
        "문서 카테고리": "인수합병",
        "요약": "삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.",
        "주요 이벤트": ["기업 인수합병", "신제품 출시"]
    }
]

# data 내에 '요약'만 추출해서 리스트로 반환
doc_list = [item['요약'] for item in data]

# elastic search based retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list, metadatas=[{"source": i} for i in range(len(data))]
)
bm25_retriever.k = 1

# embedding from openai
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# vector database retriever
## FAISS
faiss_vectorstore = FAISS.from_texts(
    doc_list, embedding=embedding, metadatas=[{'source': i} for i in range(len(data))]
)
faiss_retriever = faiss_vectorstore.as_retriever(
    search_kwargs={'k':1}
)
## Chroma
chroma_vectorstore = Chroma.from_texts(
    doc_list, embedding=embedding, metadatas=[{'source': i} for i in range(len(data))]
)
chroma_retriever = chroma_vectorstore.as_retriever(
    search_kwargs={'k':1}
)

# ensemble retreiver
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever, chroma_retriever],
    weights=[0.2, 0.5, 0.3]
)

def retrieve(query):
    ensemble_docs = ensemble_retriever.invoke(query)
    return ensemble_docs
    
def get_answer(query, model):
    if 'gpt' in model:
        llm = OpenAI(api_key=OPENAI_API_KEY)
        response = llm.chat.completions.create(
            model=model,
            messages=[{
                'role':'system',
                'content':'You are a helpful assistant.'
            }, {
                'role':'user',
                'content':query
            }]
        )
        answer = response.choices[0].message.content
    elif 'claude' in model:
        llm = Anthropic(api_key=ANTHROPIC_API_KEY)
        response = llm.messages.create(
            model=model,
            max_tokens=1024,
            messages=[
                {'role':'user', 'content':query},
                {'role':'assistant', 'content':"Hello!"},
            ]
        )
        answer = response.content[0].text
        
    elif 'llama' in model:
        llm = ChatOllama(model=model)
        response = llm.invoke(query)
        answer = response.content
        
    else:
        print("Not supported")
        return None
    
    return answer

def prompt_and_generate(query, docs, price, model):
    
    prompt = f"""아래 질문을 기반으로 검색된 뉴스를 참고하여 질문에 대한 답변을 생성하시오. 질문과 가장 적절한 뉴스에서 등장한 이벤트까지 출력하세요.
    답변 마지막에 오늘의 종가 정보도 포함시켜 답변을 만드시오.
    
    # 종가 : {price}
    
    # 질문 : {query}
    
    # 뉴스
    """
    
    # docs가 여러 개인 경우 나눠서 붙임
    for i in range(len(docs)):
        prompt += f"뉴스{i+1}\n"
        prompt += f"요약: {docs[i].page_content}\n"
        idx = docs[i].metadata['source']
        prompt += f"카테고리: {data[idx]['문서 카테고리']}\n"
        prompt += f"이벤트: {data[idx]['주요 이벤트']}\n"
        prompt += '\n'
        
    print(prompt)
    answer = get_answer(prompt, model=model)
    return answer

# query에서 기업 ticker 추출
def first_chain(query, model):
    prompt = """아래 질문에 기업명이 포함되어 있다면, 기업번호(ticker)를 출력하시오. 반드시 ticker만을 출력하고, 기업명이 포함되어 있지 않은 경우는 0을 출력하시오.
    
    # 출력 포맷 : {'기업명':'기업번호'}\n
    """
    prompt += f"질문: {query}"
    answer = get_answer(prompt, model)
    return answer
        

In [21]:
query = '삼성전자가 인수하는 기업은?'
model_first = 'gpt-3.5-turbo'
today = datetime.today().strftime('%Y%m%d')
today = '20240905'
model_second = 'llama3.1'

# 기업 종가 정보 추출
companies = first_chain(query, model_first)
company_dict = eval(companies)
tickers = list(company_dict.values())
prices = stock.get_market_ohlcv(today, today, tickers[0])

if len(prices) > 0:
    # 날짜와 종가 column 가져오기기
    price = prices.iloc[0, 3]
else:
    price = "unknown"
    
retrieved = [doc for doc in retrieve(query)]
answer = prompt_and_generate(query, retrieved, price, model_second)
answer

아래 질문을 기반으로 검색된 뉴스를 참고하여 질문에 대한 답변을 생성하시오. 질문과 가장 적절한 뉴스에서 등장한 이벤트까지 출력하세요.
    답변 마지막에 오늘의 종가 정보도 포함시켜 답변을 만드시오.
    
    # 종가 : 69000
    
    # 질문 : 삼성전자가 인수하는 기업은?
    
    # 뉴스
    뉴스1
요약: 삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.
카테고리: 인수합병
이벤트: ['기업 인수합병']

뉴스2
요약: 테스트 하나 둘 셋
카테고리: 인수합병
이벤트: ['신제품 출시']




'삼성전자가 인수하는 기업은?\n \n삼성전자는 HVAC(냉난방공조) 사업을 인수하기 위한 협상을 진행 중입니다. 이는 기존 가전 사업의 약점을 보완하는 목적을 가지고 있습니다. \n \n오늘의 종가 : 69000'

In [24]:
# 기업 종가 정보 통합한 대답 결과

def get_answer_final(query, model_first, model_second):
    
    # 기업명 및 기업번호 추출 후 주식의 종가 획득득
    today = datetime.today().strftime('%Y%m%d')
    today = '20240905'
    companies = first_chain(query, model_first)
    company_dict = eval(companies)
    tickers = list(company_dict.values())
    prices = stock.get_market_ohlcv(today, today, tickers[0])
    
    if len(prices) > 0:
        price = prices.iloc[0, 3]
        
    else:
        price = 'unknown'
    
    print(price)
    
    retrieved = [doc for doc in retrieve(query)]
    answer = prompt_and_generate(query, retrieved, price, model_second)
    return answer
    
    

In [25]:
query = '삼성전자가 인수하는 기업은?'
model_first = 'gpt-3.5-turbo'
model_second = 'llama3.1'
final_answer = get_answer_final(query, model_first, model_second)
final_answer

69000
아래 질문을 기반으로 검색된 뉴스를 참고하여 질문에 대한 답변을 생성하시오. 질문과 가장 적절한 뉴스에서 등장한 이벤트까지 출력하세요.
    답변 마지막에 오늘의 종가 정보도 포함시켜 답변을 만드시오.
    
    # 종가 : 69000
    
    # 질문 : 삼성전자가 인수하는 기업은?
    
    # 뉴스
    뉴스1
요약: 삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.
카테고리: 인수합병
이벤트: ['기업 인수합병']

뉴스2
요약: 테스트 하나 둘 셋
카테고리: 인수합병
이벤트: ['신제품 출시']




'삼성전자가 인수하는 기업은?\n\n뉴스를 검색 결과가 나와 삼성전자 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다는 뉴스가 등장합니다. 따라서, 삼성전자의 인수 사례는 기존 가전 사업의 약점을 보완하기 위해 진행하는 것으로 볼 수 있습니다.\n\n오늘 종가: 69,000'